# LOGISTIC REGRESSION

Xây dựng logistic model để dự đoán chuyến bay có bị trễ hay không từ các biến:  
- mon  
- dom  
- dow  
- carrier  
- org  
- km (mile * 1.60934)  
- depart  
- duration

## Nhập dữ liệu

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc= SparkContext(master= 'local', appName= 'Demo Logistic regression')
ss= SparkSession(sc)

In [35]:
path= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Data ML/flights.csv'
df= ss.read.csv(path= path, inferSchema= True, header= True)

In [4]:
df.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [29]:
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



## Chuẩn hóa dữ liệu

In [36]:
# Tạo biến km
from pyspark.sql.functions import round
df= df.withColumn('km', round(col('mile') * 1.60934, 0))

In [37]:
# Mã hóa biến delay. Nếu delay >= 15 = 1
df= df.withColumn('label', (col('delay') >= 15).cast('integer'))
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|1989.0|    0|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA| 415.0| null|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 5 rows



In [38]:
# Xóa các dòng mà label NA
df= df.dropna(how= 'any', subset= 'label')
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 5 rows



## Tạo dữ liệu train, test

In [46]:
train, test= df.randomSplit([0.8, 0.2])

## Chuyển đổi dữ liệu

### Mã hóa các biến số định danh (carrier và org)

In [53]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
# Mã hóa biến số carrier thành dạng số
carrier_indexer= StringIndexer(inputCol= 'carrier', outputCol= 'carrier_idx')
carrier_indexer= carrier_indexer.fit(train)
train= carrier_indexer.transform(train)
# Mã hóa biến số org thành dạng số
org_indexer= StringIndexer(inputCol=  'org', outputCol=  'org_idx')
org_indexer= org_indexer.fit(train)
train= org_indexer.transform(train)
# Onehot encoding
oh_encoder= OneHotEncoderEstimator(inputCols= ['carrier_idx', 'org_idx'], outputCols= ['carrier_dummy', 'org_dummy'])
oh_encoder= oh_encoder.fit(train)
train= oh_encoder.transform(train)

In [54]:
train.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|carrier_idx|org_idx|carrier_dummy|    org_dummy|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
|  0|  1|  2|     AA|    73|ORD|4243|  9.08|     560|   39|6828.0|    1|        1.0|    0.0|(8,[1],[1.0])|(7,[0],[1.0])|
|  0|  1|  2|     AA|   150|SFO|2704| 23.42|     325|   22|4352.0|    1|        1.0|    1.0|(8,[1],[1.0])|(7,[1],[1.0])|
|  0|  1|  2|     AA|   154|ORD| 867| 17.25|     135|   49|1395.0|    1|        1.0|    0.0|(8,[1],[1.0])|(7,[0],[1.0])|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
only showing top 3 rows



### Tạo vector

In [60]:
from pyspark.ml.feature import VectorAssembler
assembler= VectorAssembler(inputCols= ['mon', 'dom', 'dow', 'flight', 'depart', 'duration', 'km', 'carrier_dummy', 'org_dummy'],
                           outputCol= 'features')
train= assembler.transform(train)

In [63]:
train.select('features', 'label').show(10, False)

+------------------------------------------------------------------+-----+
|features                                                          |label|
+------------------------------------------------------------------+-----+
|(22,[1,2,3,4,5,6,8,15],[1.0,2.0,73.0,9.08,560.0,6828.0,1.0,1.0])  |1    |
|(22,[1,2,3,4,5,6,8,16],[1.0,2.0,150.0,23.42,325.0,4352.0,1.0,1.0])|1    |
|(22,[1,2,3,4,5,6,8,15],[1.0,2.0,154.0,17.25,135.0,1395.0,1.0,1.0])|1    |
|(22,[1,2,3,4,5,6,8,17],[1.0,2.0,181.0,17.0,379.0,3983.0,1.0,1.0]) |0    |
|(22,[1,2,3,4,5,6,8],[1.0,2.0,254.0,15.33,310.0,4001.0,1.0])       |1    |
|(22,[1,2,3,4,5,6,8,18],[1.0,2.0,317.0,9.92,170.0,1180.0,1.0,1.0]) |0    |
|(22,[1,2,3,4,5,6,8,18],[1.0,2.0,335.0,14.58,165.0,1180.0,1.0,1.0])|0    |
|(22,[1,2,3,4,5,6,8,15],[1.0,2.0,336.0,21.58,115.0,1180.0,1.0,1.0])|1    |
|(22,[1,2,3,4,5,6,8,15],[1.0,2.0,346.0,19.5,130.0,1180.0,1.0,1.0]) |1    |
|(22,[1,2,3,4,5,6,8,15],[1.0,2.0,354.0,17.58,130.0,1180.0,1.0,1.0])|1    |
+------------------------

### Scale dữ liệu (lựa chọn minmax scale)

In [67]:
from pyspark.ml.feature import MinMaxScaler
mm_scaler= MinMaxScaler(inputCol= 'features', outputCol= 'features_scale')
mm_scaler= mm_scaler.fit(train)
train= mm_scaler.transform(train)

## Xây dựng mô hình hồi quy Logistic trên tập train

In [71]:
from pyspark.ml.classification import LogisticRegression
lgr= LogisticRegression(featuresCol= 'features_scale', labelCol= 'label')

In [84]:
lgr= lgr.fit(train)

## Đánh giá mô hình hồi quy

### Đánh giá mô hình trên tập train

#### Đánh giá mô hình bằng phương thức evaluate thuộc class LogisticRegression  
Bằng phương thức này không cần thiết phải transform dữ liệu

In [154]:
lgr_evaluater= lgr.evaluate(dataset= train.select('features_scale', 'label'))

In [160]:
lgr_evaluater.accuracy

0.6110313281208446

In [161]:
lgr_evaluater.recallByLabel

[0.5535908596300326, 0.6659556757881594]

In [167]:
lgr_evaluater.precisionByLabel

[0.6130995420583273, 0.609397315052842]

In [168]:
lgr_evaluater.areaUnderROC

0.6534309616893683

#### Đánh giá mô hình bằng class BinaryClassificationEvaluator và MulticlassClassificationEvaluator trong modul evaluation

In [140]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [179]:
classification_evalaluator= MulticlassClassificationEvaluator(labelCol= 'label',
                                                              predictionCol= 'prediction')

In [180]:
classification_evalaluator.setMetricName('f1').evaluate(train)

0.6097362212301496

In [181]:
classification_evalaluator.setMetricName('weightedPrecision').evaluate(train)

0.6112069776335776

In [182]:
classification_evalaluator.setMetricName('weightedRecall').evaluate(train)

0.6110313281208446

In [190]:
classification_evalaluator.setMetricName('accuracy').evaluate(train)

0.6110313281208446

In [195]:
bi_classification_evalaluator= BinaryClassificationEvaluator(labelCol= 'label')

In [196]:
bi_classification_evalaluator.setMetricName('areaUnderROC').evaluate(train)

0.6534364471069379

In [197]:
bi_classification_evalaluator.setMetricName('areaUnderPR').evaluate(train)

0.6386669075509808

__Nhận xét: Đánh giá bằng phương thức evaluate cho các chỉ số thông dụng còn đánh giá mô hình phân loại bằng modul evaluation cho các chỉ số chuyên sâu, ít thông dụng hơn. Vậy nên phân tích hồi quy logistic theo hướng: sử dụng pipeline để chuyển đổi dữ liệu mà không đưa đối tượng hồi quy vào pipeline.__

## Lưu và load model

###  Lưu

In [198]:
lgr.save('Logistic regression model demo')

### Load

In [203]:
from pyspark.ml.classification import  LogisticRegressionModel
lgr_load= LogisticRegressionModel.load('Logistic regression model demo')